# sscan as Bluesky plan

## 1D step scans using sscan record

Support the [sscan record](https://epics.anl.gov/bcda/synApps/sscan/sscanRecord.html) with a  [bluesky](http://nsls-ii.github.io/bluesky) plan for data acquisition.  Consider the case of [1D step scans using sscan record](https://epics.anl.gov/bcda/synApps/sscan/sscanRecord.html#HEADING_1-1).

In [1]:
import asyncio
from collections import deque, OrderedDict
import numpy as np
import time

%matplotlib notebook
from bluesky.utils import install_qt_kicker
install_qt_kicker()

# common IOC prefix to be used
P = "prj:"

In [2]:
from ophyd.scaler import ScalerCH
scaler = ScalerCH(f"{P}scaler1", name="scaler")
scaler.select_channels(None)

In [3]:
from ophyd import EpicsMotor
m1 = EpicsMotor(f"{P}m1", name="m1")

In [4]:
from apstools.synApps_ophyd import userCalcsDevice
calcs = userCalcsDevice(P, name="calcs")

In [5]:
from apstools.synApps_ophyd import sscanDevice
scans = sscanDevice(P, name="scans")
scans.select_channels()

In [6]:
from apstools.synApps_ophyd import SaveData
save_data = SaveData(f"{P}saveData_", name="save_data")

In [7]:
# configure saveData for data collection into MDA files:
        
save_data.file_system.put("/tmp")
save_data.subdirectory.put("saveData")
save_data.base_name.put("sscan1_")
save_data.next_scan_number.put(1)
save_data.comment1.put("testing")
save_data.comment2.put("configured and run from ophyd")

In [8]:
# configure the sscan record for data collection:

# clear out the weeds
scans.reset()

scan = scans.scan1
scan.number_points.put(6)
scan.positioners.p1.setpoint_pv.put(m1.user_setpoint.pvname)
scan.positioners.p1.readback_pv.put(m1.user_readback.pvname)
scan.positioners.p1.start.put(-1)
scan.positioners.p1.end.put(0)
scan.positioner_delay.put(0.0)
scan.detector_delay.put(0.1)
scan.detectors.d01.input_pv.put(scaler.channels.chan03.s.pvname)
scan.detectors.d02.input_pv.put(scaler.channels.chan02.s.pvname)
scan.triggers.t1.trigger_pv.put(scaler.count.pvname)

# finally, reconfigure
scans.select_channels()

In [9]:
# make a noisy detector in an EPICS swait record, peak ceneter at 2
from apstools.synApps_ophyd import swait_setup_lorentzian
swait_setup_lorentzian(calcs.calc2, m1, 2)
noisy_det = calcs.calc2.val
noisy_det.kind = "hinted"

In [10]:
def ophyd_step_scan(motor):
    """step-scan the motor and read the noisy detector"""
    t0 = time.time()
    for p in range(10):
        motor.move(p-3)
        print(
            "%8.3f" % (time.time()-t0), 
            "%8.2f" % motor.position, 
            "%8.4f" % noisy_det.get()
             )
    motor.move(0)
    print("Complete in %.3f seconds" % (time.time()-t0))

# ophyd_step_scan(m1)

--------
## setup Bluesky, databroker, and the RunEngine

In [11]:
from databroker import Broker
db = Broker.named("mongodb_config")

In [12]:
from bluesky import RunEngine
import bluesky.plans as bp
import bluesky.plan_stubs as bps
from bluesky.callbacks.best_effort import BestEffortCallback
from bluesky import SupplementalData

RE = RunEngine({})
RE.subscribe(db.insert)
RE.subscribe(BestEffortCallback())
RE.preprocessors.append(SupplementalData())

------
## Develop the BS plan

In [13]:
import apstools.plans as APS_plans
import ophyd

DIAGNOSTIC_PRINTING = False


def get_sscan_data_objects(sscan):
    """
    prepare a dictionary of the "interesting" ophyd data objects for this scan
    """
    scan_data_objects = OrderedDict()
    for part in (sscan.positioners, sscan.detectors):
        for chname in part.read_attrs:
            if not chname.endswith("_value"):
                continue
            obj = getattr(part, chname)
            key = obj.name.lstrip(sscan.name + "_")
            scan_data_objects[key] = obj
    return scan_data_objects

    
def sscan_step_1D(sscan, _md={}):
    """
    simple 1-D step scan using EPICS synApps sscan record
    
    assumes the sscan record has already been setup properly for a scan
    """
    global new_data
    
    t0 = time.time()
    sscan_status = ophyd.DeviceStatus(sscan.execute_scan)
    started = False
    new_data = False
    
    def execute_cb(value, timestamp, **kwargs):
        """watch for sscan to complete"""
        if DIAGNOSTIC_PRINTING:
            elapsed = "%.3f" % (time.time() - t0)
            phase = sscan.scan_phase.get(as_string=True)
            print(
                f"{elapsed} execute_cb()"
                f" {timestamp}:"
                f" phase={phase}"
                f" executing={value}"
            )
        if started and value in (0, "IDLE"):
            sscan_status._finished()
            sscan.execute_scan.unsubscribe_all()
            sscan.scan_phase.unsubscribe_all()
    
    def phase_cb(value, timestamp, **kwargs):
        """watch for new data"""
        global new_data
        if DIAGNOSTIC_PRINTING:
            elapsed = "%.3f" % (time.time() - t0)
            phase = sscan.scan_phase.enum_strs[value]
            print(
                f"{elapsed} phase_cb()"
                f" {timestamp}:"
                f" phase={phase}"
                f" value={value}"
            )
        if value in (15, "RECORD SCALAR DATA"):
            new_data = True            # set flag for main plan
    
    sscan.select_channels()
    sscan_data_objects = get_sscan_data_objects(sscan)
    
    sscan.execute_scan.subscribe(execute_cb)
    sscan.scan_phase.subscribe(phase_cb)

    yield from bps.open_run(_md)               # start data collection
    yield from bps.mv(sscan.execute_scan, 1)   # start sscan
    started = True

    # collect and emit data, wait for sscan to end
    while not sscan_status.done or new_data:
        if DIAGNOSTIC_PRINTING:
            elapsed = "%.3f" % (time.time() - t0)
            print(
                f"{elapsed} plan()"
                f" new data={new_data}"
                f" status={sscan_status}"
            )
        if new_data:
            new_data = False
            yield from bps.create("primary")
            for k, obj in sscan_data_objects.items():
                if DIAGNOSTIC_PRINTING:
                    print(f"obj[{k}] = {obj}")
                yield from bps.read(obj)
            yield from bps.save()
        yield from bps.sleep(0.001)

    # dump the entire sscan record into another stream
    yield from bps.create("sscan")
    yield from bps.read(sscan)
    yield from bps.save()

    yield from bps.close_run()

    return sscan_status

In [14]:
scans = sscanDevice(P, name="scans")
scans.select_channels()

RE(sscan_step_1D(scans.scan1), md=dict(purpose="development", issue="#91"))

Transient Scan ID: 1     Time: 2019/03/12 06:26:00
Persistent Unique Scan ID: 'c09258e8-04a3-47d5-87b9-a016044c3bb9'
New stream: 'primary'
+-----------+------------+
|   seq_num |       time |
+-----------+------------+
|         1 | 06:26:02.2 |
|         2 | 06:26:03.0 |
|         3 | 06:26:03.9 |
|         4 | 06:26:04.9 |
|         5 | 06:26:05.8 |
|         6 | 06:26:06.7 |
New stream: 'sscan'
+-----------+------------+
generator sscan_step_1D ['c09258e8'] (scan num: 1)





('c09258e8-04a3-47d5-87b9-a016044c3bb9',)

In [15]:
h = db[-1]

In [16]:
h.table()

,time,scans_scan1_positioners_p1_readback_value,scans_scan1_detectors_d01_current_value,scans_scan1_detectors_d02_current_value,scans_scan1_positioners_p1_setpoint_value
seq_num,,,,,
1,2019-03-12 06:26:02.209883928,-1.0,1.0,0.0,-8.000000e-01
2,2019-03-12 06:26:03.098092794,-0.8,1.0,1.0,-6.000000e-01
3,2019-03-12 06:26:03.999228239,-0.6,2.0,2.0,-4.000000e-01
4,2019-03-12 06:26:04.903812647,-0.4,5.0,4.0,-2.000000e-01
5,2019-03-12 06:26:05.809070110,-0.2,1.0,1.0,-5.551115e-17
6,2019-03-12 06:26:06.706605196,0.0,1.0,0.0,-5.551115e-17


In [17]:
h.table("sscan")

,time,scans_scan1_scan_phase,scans_scan1_data_state,scans_scan1_data_ready,scans_scan1_scan_busy,scans_scan1_alert_flag,scans_scan1_alert_message,scans_scan1_current_point,scans_scan1_pasm,scans_scan1_execute_scan,...,scans_scan1_acqm,scans_scan1_atime,scans_scan1_copyto,scans_scan1_a1cd,scans_scan1_ascd,scans_scan1_positioners_p1_readback_value,scans_scan1_positioners_p1_setpoint_value,scans_scan1_detectors_d01_current_value,scans_scan1_detectors_d02_current_value,scans_scan1_triggers_t1_trigger_value
seq_num,,,,,,,,,,,,,,,,,,,,,
1,2019-03-12 06:26:06.773308992,0,7,1,0,0,SCAN Complete,6,0,0,...,0,0.0,0,1.0,1.0,0.0,-5.551115e-17,1.0,0.0,1.0
